<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from glob import glob
from sklearn.model_selection import train_test_split

# **CallBack**

In [2]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.70):
            print("\nReached 70.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [6]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

fatal: destination path 'skin-disease-build' already exists and is not an empty directory.


In [19]:
train_dir = r"C:\Users\Aziz Shaikh\Desktop\mywork\skin-disease-build\dataset\train"
test_dir = r"C:\Users\Aziz Shaikh\Desktop\mywork\skin-disease-build\dataset\test"

In [24]:
assert len(os.listdir(train_dir)) == 23 and len(os.listdir(test_dir)) == 23

# **Image Generator**

In [25]:
data_generator = ImageDataGenerator(validation_split = 0.2)

In [26]:
train_datagen = ImageDataGenerator(rescale=1./255,
#                                   rotation_range=40,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
                                  horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)
#                                   rotation_range=40
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True)
                                 

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=64,
                                                   class_mode='sparse',
                                                   target_size=(200,200))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( test_dir,
                                                   batch_size=64,
                                                   class_mode='sparse',
                                                   target_size=(200,200))


Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


## Model

In [42]:
i = Input(shape = (200,200,3))
x = Conv2D(16,(3,3),activation='relu')(i)
x = MaxPooling2D((2,2))(x)
#x = BatchNormalization()(x)
x = Conv2D(32,(3,3),activation='relu')(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
#   x = Dropout(0.25)(x)
x = Conv2D(64,(3,3),activation='relu')(x)
#x = BatchNormalization()(x)
x = Conv2D(128,(3,3),activation='relu')(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
#   x = Dropout(0.25)(x)
# x = Conv2D(128,(3,3),activation='relu')(x)
# x = BatchNormalization()(x)
# x = Conv2D(128,(3,3),activation='relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(23,activation='softmax')(x)

In [43]:
model = Model(i,x)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 198, 198, 16)      448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 46, 46, 64)        18496     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 44, 44, 128)       7385

In [44]:
model.compile(optimizer = Adam(),
          loss = 'sparse_categorical_crossentropy',
          metrics = ['sparse_categorical_accuracy'])

In [45]:
history = model.fit_generator(train_generator,validation_data=validation_generator,
                              epochs=30,steps_per_epoch = train_generator.n//train_generator.batch_size,
                              validation_steps = 64)

Epoch 1/30
243/243 [==============================] - 843s 3s/step - loss: 2.9108 - sparse_categorical_accuracy: 0.1328 - val_loss: 2.8173 - val_sparse_categorical_accuracy: 0.1621
Epoch 2/30
243/243 [==============================] - 979s 4s/step - loss: 2.7605 - sparse_categorical_accuracy: 0.1821 - val_loss: 2.7042 - val_sparse_categorical_accuracy: 0.1877
Epoch 3/30
243/243 [==============================] - 1028s 4s/step - loss: 2.6616 - sparse_categorical_accuracy: 0.2121 - val_loss: 2.6289 - val_sparse_categorical_accuracy: 0.2243
Epoch 4/30
243/243 [==============================] - 914s 4s/step - loss: 2.5441 - sparse_categorical_accuracy: 0.2386 - val_loss: 2.6026 - val_sparse_categorical_accuracy: 0.2334
Epoch 5/30
243/243 [==============================] - 868s 4s/step - loss: 2.4544 - sparse_categorical_accuracy: 0.2684 - val_loss: 2.6224 - val_sparse_categorical_accuracy: 0.2388
Epoch 6/30
243/243 [==============================] - 826s 3s/step - loss: 2.3415 - sparse_cat

KeyboardInterrupt: 

## **GRAPH**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()